In [2]:
import pandas as pd
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp


def extremity_cluster_counting(user_n_cluster, user_affinity, user_linkage, depends_arr):
    cluster = AgglomerativeClustering(n_clusters=user_n_cluster, affinity=user_affinity, linkage=user_linkage)
    cluster_result = cluster.fit_predict(depends_arr)
    
    return cluster_result


def alvin_god(user_n_cluster, user_affinity, user_linkage, dir_arr, depends_arr, file_version, main_dict):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(dir_arr)

    true_k = user_n_cluster
    model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
    model.fit(X)

    #print("Top terms per cluster:")
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    
    main_cluster_arr = []
    col_counter = 0

    
    cluster = AgglomerativeClustering(n_clusters=user_n_cluster, affinity=user_affinity, linkage=user_linkage)
    cluster_result = cluster.fit_predict(depends_arr)

    cluster_result_arr = []
    cluster_counter = 0
    for element in cluster_result:
        #print(element)
        cluster_result_arr.append(element)

    main_cluster_arr.append(cluster_result_arr)
    
    col_counter += 1
    
    dir_arr_int = []
    #for element in dir_arr:
    #    dir_arr_int.append(main_dict[element])
    
    #cluster_result_int = []
    #for element in cluster_result:
        #cluster_result_int.append(main_dict[main_dict_val[element]])
    
    #print(dir_arr_int)
    #print('fk')
    #print(cluster_result_int)
    #common_results_rsf = common_member(dir_arr_int,cluster_result_int)
    #print(len(common_results_rsf))
    
    
    filename1 = 'MoJo_1.2.1/' + project_name + '/' +str(file_version) + '_' + str(user_n_cluster) + '_' + str(user_affinity) + "_" + str(user_linkage) + '_b' +".rsf"
    #print(filename1)
    with open(filename1, 'w') as f:
        for i in range(len(dir_arr)):
            Y = vectorizer.transform([dir_arr[i]])
            string = "contain " + str(model.predict(Y)[0]) + " " +str(i) + "\n"
            f.write(string)

        f.close()
    
    filename2 = 'MoJo_1.2.1/' + project_name + '/' +str(file_version) + '_' + str(user_n_cluster) + '_' + str(user_affinity) + "_" + str(user_linkage) + '_a' +".rsf"
    with open(filename2, 'w') as f:
        for i in range(len(cluster_result)):
            #print(element)
            string = "contain " + str(cluster_result[i]) + " " + str(i) + "\n"
            f.write(string)
        f.close()
        #for element in cluster_result:
            #print(element)
            #string = "contain " + str(element) + " " + str(main_dict[main_dict_val[element]]) + "\n"
            #f.write(string)
        #f.close()

    #print('Difference is: ' + str(len(dir_arr) - len(G.nodes) ))
    if len(dir_arr) - len(G.nodes) > 0:
        with open(filename2,'a') as f:
            for i in range(len(dir_arr)-len(G.nodes)):
                tbc = (i+len(G.nodes))
                string = "contain " + str(tbc) + " " + str(tbc) + "\n"
                f.write(string)

            f.close()
    else:
        with open(filename1,'a') as f:
            for i in range(len(G.nodes) - len(dir_arr)):
                tbc = i+len(dir_arr)
                string = "contain " + str(tbc) + " " + str(tbc) + "\n"
                f.write(string)
            f.close()

    return filename1, filename2




def fileIsSame(right, left, path):
    return os.path.exists (os.path.join(left, path.replace(right, '')));

def compare(right, left):
    difference = list();
    for root, dirs, files in os.walk(right):
        for name in files:
            path = os.path.join(root, name);
            # check if file is same
            if fileIsSame(right, left, path):
                if os.path.isdir(path):
                    # recursively check subdirs
                    difference.extend(compare(path, left));
            else:
                # count file as difference
                difference.append(path);

    return difference;

def compare_similar(right, left):
    difference = list();
    for root, dirs, files in os.walk(right):
        for name in files:
            path = os.path.join(root, name);
            # check if file is same
            if fileIsSame(right, left, path):
                
                if os.path.isdir(path):
                    # recursively check subdirs
                    difference.extend(compare_similar(path, left));
                difference.append(path)
                
            #else:
                # count file as difference
                #difference.append(path);

    return difference;



def common_member(a,b):
    a_set = set(a)
    b_set = set(b)
    if len(a_set.intersection(b_set)) > 0:
        return(a_set.intersection(b_set))
    else:
        return set()

In [3]:
results = pd.read_csv('best_result_agglomerative_75_ver.csv')

In [4]:
results['extremity'] = 0

In [5]:
results.head()

,name,version,n_clusters,affinity,linkage,n_MoJo,mojoFM,extremity
0,apache-spark,2.4.3,50,l1,single,107,99.228217,0
1,jenkinsci-jenkins,2.194,50,cosine,single,114,99.020029,0
2,ionic-team-capacitor,0.0.112,50,l2,single,8,99.008674,0
3,hibernate-hibernate-orm,5.3.7,50,manhattan,single,111,98.986301,0
4,iSoron-uhabits,1.7.8,300,l1,complete,9,98.604651,0


In [9]:
project_names = []
for element in results.sort_values(['name'], ascending=True)['name']:
    #print(element)
    project_names.append(element)

In [12]:
#project_names = ['apache-jmeter']

for project_name in project_names:
    
    release_arr = []
    ver = pd.read_csv('project_links/' +project_name + '.csv', encoding='ISO-8859-1')
    ver.head()
    
    link = open('project_links/' + project_name + '.txt')
    for line in link:
        link_str = line

    link.close()
    for index, row in ver.iterrows():
    #print(index)
    #print('lol')
    #print(row['release_tag'])
        release_arr.append((row['release_tag'], row['version_name']))
    '''
    element = release_arr[0]
    command = 'cd C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/' + project_name + ' & mkdir ' + project_name + '-' + element[1]
    print(command)
    os.system(command)
    
    command = 'git clone ' + link_str +  ' C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/' + project_name + '/' + project_name + '-' +element[1]
    print(command)
    os.system(command)
    
    command = 'cd C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/' + project_name + '/' + project_name +'-' + element[1] + ' & git checkout ' + element[0]
    print(command)
    os.system(command)
    print(element)
    
    rootdir = 'raw_sourcecode'
    dir_arr = []
    rootdir_arr = []
    depth=2

    
    for root, dirs, files in os.walk(rootdir):
        if root.count(os.sep) == depth and project_name in str(root):
            #print(root)
            dir_arr.append(root)
            rootdir_arr.append(root)
            
    os.mkdir('MoJo_1.2.1/' + project_name)
    os.mkdir('raw_depends/' + project_name)
    for i in range(len(dir_arr)):
        #print(element)
        command = 'cd C:/Users/tanji/Desktop/FIT4003/depends-0.9.2 & ' + 'java -jar depends.jar java C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/'  + project_name + '/' + dir_arr[i].split('\\')[-1] +  ' ../raw_depends/' + project_name + '/' + str(i) + '_' + dir_arr[i].split('\\')[-1] 
        #print(command)
        #os.system('cd C:/Users/tanji/Desktop/FIT4003/MoJo_1.2.1 & java MoJo test/test2_a.rsf test/test2_b.rsf > test/test_results.txt')
        os.system(command)
    

    rootdir = 'raw_depends/' + project_name
    
    for root, dirs, files in os.walk(rootdir):

        #print(files)
        for element in files:

            with open(rootdir + '/' + element) as f:
                spark_results = json.load(f)
                
                main_dict = {}
                main_dict_val = {}
                main_dict_counter = 0
                dict_array = []
                min_set = set()
                for element in spark_results['variables']:
                    #print(element)

                    ### Replace this with the initial JSON directory ###
                    #print(json_name_dict[str(i+10)])
                    element_tbc = element.replace('C:\\Users\\tanji\\Desktop\\FIT4003\\raw_sourcecode\\' + project_name + '\\' +results.loc[results['name']=='apache-spark']['version'][0] , '')
                    #print(element)
                    main_dict[element_tbc] = main_dict_counter
                    main_dict_val[main_dict_counter] = element_tbc
                    main_dict_counter += 1


                index  = 0
                var_array = []
                for value in spark_results['variables']:
                    var_array.append([index, value])
                    #print(index, value)
                    index += 1

                var_df = pd.DataFrame(var_array)
                var_df.columns = ['index_val', 'name']

                feature_list = {}
                feature_index = 2
                for element in spark_results['cells']:
                    #print(element)
                    try:
                        for a in element['values']:
                            if a not in feature_list:
                                feature_list[a] = feature_index
                                feature_index += 1
                            #print(a['Call'])


                    except:
                        pass

                feature_arr = []
                for element in spark_results['cells']:
                    #print(array)
                    array = [0] * (len(feature_list) + 2)
                    values = dict(element['values'])
                    #print(element)
                    array[0] = element['src']
                    array[1] = element['dest']
                    for feature in feature_list:
                        try:
                            value = values[feature]
                            array[feature_list[feature]] = value
                        except:
                            pass
                    #print(array)
                    feature_arr.append(array)

                feature_df = pd.DataFrame(feature_arr)
                col_names = ['src', 'dest']
                for element in feature_list:
                    col_names.append(element)
                feature_df.columns = col_names

                feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']
                G = nx.Graph()
                for index, row in feature_df.iterrows():
                    G.add_edge(row['src'], row['dest'], weight=row['sum'])

                adj_mat = nx.adjacency_matrix(G)
                adj_mat_df = pd.DataFrame(adj_mat.todense())
                np.fill_diagonal(adj_mat_df.values, adj_mat_df.values.max())
                x = adj_mat_df.values
                min_max_scaler = preprocessing.MinMaxScaler()
                x_scaled = min_max_scaler.fit_transform(x)
                adj_mat_df = pd.DataFrame(x_scaled)

                #os.mkdir('MoJo_1.2.1/' + project_name)

                filename_arr = []
                number = results.loc[results['name']=='apache-spark']['n_clusters'][0]
                affinity_type = results.loc[results['name']=='apache-spark']['affinity'][0]
                linkage_type = results.loc[results['name']=='apache-spark']['linkage'][0]
                depends_arr = adj_mat_df
                clustering_result = extremity_cluster_counting(number, affinity_type, linkage_type, depends_arr)
                max_num_cluster = clustering_result.max()
                clustering_extremity_array = [0] * (max_num_cluster + 1)
                for element in clustering_result:
                    clustering_extremity_array[element] += 1
                extremity_score = 0
                for element in clustering_extremity_array:
                    if element < 5 or element > 100:
                        extremity_score += 1
                final_extremity_score = 1 - (extremity_score/(max_num_cluster+1))
                results.loc[results.name == project_name, 'extremity'] = final_extremity_score
        
    '''
            
    

In [52]:
max_num_cluster = clustering_result.max()
clustering_extremity_array = [0] * (max_num_cluster + 1)
for element in clustering_result:
    clustering_extremity_array[element] += 1
    
extremity_score = 0
for element in clustering_extremity_array:
    if element < 5 or element > 100:
        extremity_score += 1
final_extremity_score = 1 - (extremity_score/(max_num_cluster+1))

print(extremity_score)
print(max_num_cluster)

print(final_extremity_score)

50
49
0.0


In [46]:
clustering_extremity_array

[1226,
 4,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [19]:
dir_arr

['raw_sourcecode\\apache-jmeter\\apache-jmeter-5.1.1']

In [23]:
tmp

{'schemaVersion': '1.0',
 'name': '../raw_depends/apache-jmeter/0_apache-jmeter-5.1.1-sdsm',
 'variables': ['C:\\Users\\tanji\\Desktop\\FIT4003\\raw_sourcecode\\apache-jmeter\\apache-jmeter-5.1.1\\src\\components\\org\\apache\\jmeter\\assertions\\BSFAssertion.java',
  'C:\\Users\\tanji\\Desktop\\FIT4003\\raw_sourcecode\\apache-jmeter\\apache-jmeter-5.1.1\\src\\components\\org\\apache\\jmeter\\assertions\\BSFAssertionBeanInfo.java',
  'C:\\Users\\tanji\\Desktop\\FIT4003\\raw_sourcecode\\apache-jmeter\\apache-jmeter-5.1.1\\src\\components\\org\\apache\\jmeter\\assertions\\BeanShellAssertion.java',
  'C:\\Users\\tanji\\Desktop\\FIT4003\\raw_sourcecode\\apache-jmeter\\apache-jmeter-5.1.1\\src\\components\\org\\apache\\jmeter\\assertions\\CompareAssertion.java',
  'C:\\Users\\tanji\\Desktop\\FIT4003\\raw_sourcecode\\apache-jmeter\\apache-jmeter-5.1.1\\src\\components\\org\\apache\\jmeter\\assertions\\CompareAssertionBeanInfo.java',
  'C:\\Users\\tanji\\Desktop\\FIT4003\\raw_sourcecode\\apac

In [110]:
results.loc[results['name']==project_name]

,name,version,n_clusters,affinity,linkage,n_MoJo,mojoFM,extremity
54,apache-jmeter,2.7,50,euclidean,single,266,81.97832,0.44


In [31]:
print(results.loc[results['name']=='apache-spark']['n_clusters'][0])
print(results.loc[results['name']=='apache-spark']['affinity'][0])
print(results.loc[results['name']=='apache-spark']['linkage'][0])

50
l1
single


In [109]:
print(final_extremity_score)
print(project_name)
results.loc[results.name == project_name, 'extremity'] = final_extremity_score

0.44
apache-jmeter
